In [1]:
# --- Step 1: Import Libraries ---
import pandas as pd
from itertools import combinations
import os
import platform
from math import ceil
import subprocess
from tqdm import tqdm

In [2]:
# --- Step 2: Define Input & Output Paths ---
current_folder = os.getcwd()
input_file = os.path.join(current_folder, r"C:\Users\Swarupa\Downloads\50 comp full data file.csv")  
output_folder = os.path.join(current_folder, "Batched_Output_data")
os.makedirs(output_folder, exist_ok=True)

In [3]:

# --- Step 3: Read CSV ---
print(" Reading input file...")
df = pd.read_csv(input_file)
df.columns = df.columns.str.strip()

 Reading input file...


In [4]:
# --- Step 4: Identify date columns ---
date_columns = [col for col in df.columns if not col.lower().startswith("company") and "quantity" not in col.lower()]
latest_date = date_columns[0]
print(f" Using latest date column: {latest_date}")
print(f" Total date columns found: {len(date_columns)}")


 Using latest date column: 19-Aug-25
 Total date columns found: 1401


In [5]:
# --- Step 5: Compute constant Quantity per company ---
df["Quantity"] = 50000 / df[latest_date]


In [6]:
# --- Step 6: Prepare lookups ---
company_list = df["Company Name"].tolist()
price_data = df.set_index("Company Name")[date_columns]
quantity_lookup = df.set_index("Company Name")["Quantity"].to_dict()

In [7]:
# --- Step 7: Settings ---
BATCH_SIZE = 17000
all_combos = list(combinations(company_list, 4))
total_batches = ceil(len(all_combos) / BATCH_SIZE)
print(f" Total combinations: {len(all_combos)} → {total_batches} batches")


 Total combinations: 230300 → 14 batches


In [8]:
# --- Step 8: Process Each Batch ---
for batch_num in tqdm(range(total_batches), desc=" Processing Batches", unit="batch"):
    start = batch_num * BATCH_SIZE
    end = start + BATCH_SIZE
    batch_combos = all_combos[start:end]
    results = []

    # progress bar for combos in the current batch
    for combo in tqdm(batch_combos, leave=False, desc=f"Batch {batch_num + 1}/{total_batches}", unit="combo"):
        try:
            # Get constant quantities for these 4 companies
            q = [quantity_lookup[c] for c in combo]
            combo_prices = price_data.loc[list(combo)]

            # Total value for each date = Σ(q_i × price_i)
            total_values = (combo_prices.mul(q, axis=0)).sum()

            # --- Compute Backward (future-looking) 50-day moving average ---
            reversed_series = pd.Series(total_values.values[::-1])  # reverse order
            backward_ma = reversed_series.rolling(window=50, min_periods=50).mean()
            ma_series = backward_ma.values[::-1]  # reverse back to original order

            # Create one row for this combination
            row = {"Combo": ", ".join(combo)}

            # Add Total Value columns horizontally
            for i, date in enumerate(date_columns):
                row[date] = total_values.values[i]

            # Add Backward Moving Average columns horizontally
            for i, date in enumerate(date_columns):
                row[f"MA_{date}"] = ma_series[i]

            results.append(row)
        except KeyError:
            continue  # skip missing data

    # --- Step 9: Save Each Batch to Excel ---
    batch_df = pd.DataFrame(results)
    batch_file = os.path.join(output_folder, f"Batch_{batch_num + 1}.xlsx")
    batch_df.to_excel(batch_file, index=False)
    tqdm.write(f" Saved: {batch_file}")




 Processing Batches:   7%|███▉                                                   | 1/14 [16:56<3:40:16, 1016.62s/batch]

 Saved: C:\Users\Swarupa\Batched_Output_data\Batch_1.xlsx



 Processing Batches:  14%|███████▊                                               | 2/14 [34:37<3:28:31, 1042.60s/batch]

 Saved: C:\Users\Swarupa\Batched_Output_data\Batch_2.xlsx



 Processing Batches:  21%|███████████▊                                           | 3/14 [52:03<3:11:25, 1044.10s/batch]

...

 Saved: C:\Users\Swarupa\Batched_Output_data\Batch_12.xlsx



 Processing Batches:  93%|██████████████████████████████████████████████████▏   | 13/14 [3:41:37<16:55, 1015.49s/batch]

 Saved: C:\Users\Swarupa\Batched_Output_data\Batch_13.xlsx



 Processing Batches: 100%|███████████████████████████████████████████████████████| 14/14 [3:50:23<00:00, 987.41s/batch]

 Saved: C:\Users\Swarupa\Batched_Output_data\Batch_14.xlsx


In [11]:
 #--- Step 9: Auto-open Output Folder ---
print("\n All batches processed and saved successfully!")

try:
    sys_name = platform.system()
    if sys_name == "Windows":
        os.startfile(output_folder)
    elif sys_name == "Darwin":
        subprocess.Popen(["open", output_folder])
    elif sys_name == "Linux":
        subprocess.Popen(["xdg-open", output_folder])
except Exception as e:
    print(f" Could not auto-open folder: {e}")

print(f" Output folder: {output_folder}")



 All batches processed and saved successfully!
 Output folder: C:\Users\Swarupa\Batched_Output_data
